# Bert implementation

#### This ipynb is mostly for debugging BERT. Run the shell scripts for BERT as batch jobs in the HPC instead.

In [ ]:
#!pip install kagglehub
import kagglehub
import os
import shutil
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

#from datasketch import MinHash, MinHashLSH

#set working dir to be one folder up
os.chdir('..')

# Load Data

In [2]:
def load_data():
    # Contains movieId, title, genres   
    ml_movies = pd.read_csv('data/ml-32m/movies.csv')
    
    # Contains userId, movieId, rating, timestamp
    ml_ratings = pd.read_csv('data/ml-32m/ratings.csv')
    
    # Contains userId, movieId, tag, timestamp
    ml_tags = pd.read_csv('data/ml-32m/tags.csv')

    # Contains movieId, imdbId, tmdbId
    ml_links = pd.read_csv('data/ml-32m/links.csv')

    # IMDB Dataset:
    imdb = pd.read_csv('data/imdb/movies.csv')

    return ml_movies, ml_ratings, ml_tags, ml_links, imdb

ml_movies, ml_ratings, ml_tags, ml_links, imdb = load_data()

/tmp/ipykernel_3083372/683732697.py:15: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('data/imdb/movies.csv')


In [3]:
# Preprocessing IMDB:

#drop if description is "Add a plot" because we have no description then
imdb_descriptions = imdb[imdb['description'] != "Add a Plot"]

imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')

#keep what we need
imdb_descriptions = imdb_descriptions[['id', 'description']]

#merge them, so we have a li nk between the two datasets
merged_links = ml_links.merge(imdb_descriptions, left_on='imdbId', right_on='id', how='inner').dropna(subset=['id'])

/tmp/ipykernel_3083372/493108502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
/tmp/ipykernel_3083372/493108502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')


In [4]:
#merge the movies with the descriptions if that is what we want
ml_movies_description = ml_movies.merge(merged_links, left_on='movieId', right_on='movieId', how='inner')

ml_movies_description
#ml_movies_description.to_csv('data/movies_with_description.csv', index=False)

,movieId,title,genres,imdbId,tmdbId,id,description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,114709,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,113497,When two kids find and play a magical board ga...
2,6,Heat (1995),Action|Crime|Thriller,113277,949.0,113277,A group of high-end professional thieves start...
3,8,Tom and Huck (1995),Adventure|Children,112302,45325.0,112302,Two best friends witness a murder and embark o...
4,9,Sudden Death (1995),Action,114576,9091.0,114576,A former fireman takes on a group of terrorist...
...,...,...,...,...,...,...,...
51855,292541,The Settlers (2023),Drama|Western,10370812,989589.0,10370812,"A mixed-race Chilean, rides south on an expedi..."
51856,292585,Night Crawlers (2009),Comedy|Horror,985060,147230.0,985060,Blood is thicker than water in this tiny Texas...
51857,292605,Our River... Our Sky (2023),Drama|War,10676126,855800.0,10676126,Baghdad. The last week of 2006. All over the c...
51858,292613,Freelance (2023),Action|Comedy,15744298,897087.0,15744298,An ex special forces operator takes a job to p...


# Define model and tokenize

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
inputs = tokenizer(ml_movies_description['description'].tolist(), return_tensors="pt", padding=True, truncation=True)

# Run through model

In [9]:
import torch
from tqdm import tqdm
outputs = []
model = model.to("cuda")
inputs = inputs.to("cuda")
batch_size = 64
#without saving gradients to avoid gpu memory issues/leaks
# if outputs file exists, load it, else run the model
if os.path.exists('/work3/s204161/embeddings.npy'):
    outputs = torch.load('/work3/s204161/embeddings.npy')
else:
    with torch.no_grad():
        for i in tqdm(range(0, len(inputs['input_ids']), batch_size)):
            batch = inputs['input_ids'][i:i+batch_size]
            attention_mask = inputs['attention_mask'][i:i+batch_size]
            output = model(batch, attention_mask=attention_mask).last_hidden_state
            #output = model(batch, attention_mask=attention_mask)['logits']

            #send the output to the cpu afterwards to avoid memory issues on the gpu
            outputs.append(output.detach().cpu())

    #make list of tensors into one tensor
    outputs = torch.cat(outputs)
    np.save('/work3/s204161/embeddings.npy', outputs.numpy())

print("done...")
embeddings = outputs

  0%|          | 0/406 [00:00<?, ?it/s]

100%|██████████| 406/406 [01:12<00:00,  5.58it/s]


In [8]:
from sklearn.cluster import MiniBatchKMeans

# dont standardize embeddings vector to have mean 0 and std 1 ... because for our hidden weights, some features ARE more important than others...
# embeddings = embeddings - embeddings.mean(axis=0)
# embeddings = embeddings / embeddings.std(axis=0)

# Define MiniBatch K-Means
mini_batch_kmeans = MiniBatchKMeans(n_clusters=200, batch_size=1000, random_state=42)
mini_batch_kmeans.fit(embeddings)

# Get cluster labels
import copy
labels = copy.deepcopy(mini_batch_kmeans.labels_)
centers = copy.deepcopy(mini_batch_kmeans.cluster_centers_)


/work3/s204161/miniconda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
## for sparse clusters of length < 5 join them with neighbouring clusters
# get the number of movies in each cluster
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

# get the number of clusters
n_clusters = len(cluster_counts)
from tqdm import tqdm

# merge clusters with less than 5 movies
for cluster, count in tqdm(list(cluster_counts.items())):  # Iterate over a copy of the dictionary
    if count < 5:
        while True:  # Find a valid cluster to merge into
            distances = np.linalg.norm(centers - centers[cluster], axis=1)
            distances[cluster] = np.inf  # Avoid merging a cluster with itself
            closest_cluster = np.argmin(distances)
            
            # Check if the closest cluster is valid
            if closest_cluster in cluster_counts:
                break  # Exit the loop once a valid cluster is found
            
            # If not valid, continue searching
            distances[closest_cluster] = np.inf

        # merge the clusters
        labels[labels == cluster] = closest_cluster
        
        # update the cluster counts
        cluster_counts[closest_cluster] += count
        cluster_counts.pop(cluster)
        n_clusters -= 1
np.save("data/bert_clusters.npy",labels)

100%|██████████| 200/200 [00:17<00:00, 11.69it/s]


In [112]:
from scipy.spatial.distance import jaccard as jacc_score

hac =[]
for out in embeddings:#[np.argwhere(labels==20)]:
    #calculate the jaccard similarity between the first output and the rest
    jaccard = jacc_score(embeddings[0], out)
    hac.append(jaccard)

In [113]:
pd.set_option('display.max_colwidth', None)
ml_movies_description.iloc[0]

movieId                                                                                                                              1
title                                                                                                                 Toy Story (1995)
genres                                                                                     Adventure|Animation|Children|Comedy|Fantasy
imdbId                                                                                                                          114709
tmdbId                                                                                                                           862.0
id                                                                                                                              114709
description    A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.
Name: 0, dtype: object

In [4]:
#load in numpy file
import numpy as np
clusters = np.load('../data/bert_clusters.npy')
#add clusters to df
len(clusters)
#ml_movies_description['cluster_label'] = clusters
#save df
# ml_movies_description.to_csv('data/df_clusters.csv', index=False)

51860